# Analysing detection of faces with trackers

## Importing libraries

In [28]:
import gc
import os
import sys
import math
import glob
import tqdm
import random
import numpy as np
from tqdm import tqdm
import time
from time import sleep

In [29]:
import pandas as pd
import xml.etree.ElementTree as ET

In [30]:
import cv2
import dlib
from imutils import face_utils

## Defining Paths

In [31]:
path_folder_videos = '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_videos/all/'

In [32]:
path_folder_of_folders = '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/'

In [33]:
path_xml_dlib_output = 'output_XML/dlib_hog_output_232_folders.xml'

In [34]:
path_sample_video = path_folder_videos + "VIDEO_male-delight2_20180517_111853_1366476353.mp4"

In [35]:
path_sample_video_folder = path_folder_of_folders + "VIDEO_male-delight2_20180517_111853_1366476353 (5-27-2018 5-44-26 AM)/"

In [36]:
path_sample_video_folder2 = path_folder_of_folders + "VIDEO_1526475571181 bored_20180516_103925_356651116 (5-29-2018 10-06-12 AM)"
path_sample_video_folder2

'/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_1526475571181 bored_20180516_103925_356651116 (5-29-2018 10-06-12 AM)'

## Reading the XML file and getting the boxes and frame numbers in which face is detected

In [37]:
def getDictVideoToFrameNoBoxFromXML(path_xml_dlib_output):
    """
    Returns the dictionary {Video_name: {frame_1:[box_1 ie l,t,w,h], frame_2:[box_2, box_3]}} from XML file.
    """
    
    dictBox = {}
    
    #read the XML file
    tree = ET.parse(path_xml_dlib_output)
    root = tree.getroot()
    
    for image in tqdm(root.iter('image')):
        name_file = image.attrib['file']
        name_video = name_file.split('.')[0].rsplit(' ',1)[0]
        num_frame = (int)(name_file.split('.')[0].rsplit(' ',1)[1])
        
        list_boxes = []
        if (len(image) == 0):
            list_boxes = []
        else:
            for box_num,box in enumerate(image):
                box_attribs = box.attrib
                box_data = [box_attribs['left'], box_attribs['top'],box_attribs['width'],box_attribs['height']]
                list_boxes.append(box_data)
        
        if name_video in dictBox:
            dictBox[name_video][num_frame] = list_boxes
        else:
            dictBox[name_video] = {num_frame : list_boxes}
            
    return dictBox
        
        

## Detecting faces

In [38]:
def detectFaceFromPath(impath, detector):
    """
    Given the path of an image, and the detector, reads the image in grayscale and returns the bounding box rectangle
    in the format [(x,y,w,h)]
    """
    
    # load the input image, resize it, and convert it to grayscale
    image = cv2.imread(impath,0)
    
    # detect faces in the grayscale image
    rects = detector(gray, 1)
    
    result = []
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
#         shape = predictor(gray, rect)
#         shape = face_utils.shape_to_np(shape)
#         print(shape.shape)
        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)], then draw the face bounding box
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        result.append((x,y,w,h))
    
    return result
#         cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # show the face number
#         cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
#         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
#         basePath=imageTouse.split('.')[0]
#         lmpath=basePath+"_landmarks"+str(i+1)+".txt"
#         text_file = open(lmpath, "w")
#         for (x, y) in shape:
#             cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
#             text_file.write(str(x)+' '+str(y)+'\n')
#         text_file.close()


In [39]:
def detectFaceFromGrayImage(gray_image, detector):
    """
    Given the gray_image, and the detector, reads the image in grayscale and returns the bounding box rectangle
    in the format [(x,y,w,h)]
    """
    # detect faces in the grayscale image
    rects = detector(gray_image, 1)
    
    result = []
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
#         shape = predictor(gray, rect)
#         shape = face_utils.shape_to_np(shape)
#         print(shape.shape)
        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)], then draw the face bounding box
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        result.append((x,y,w,h))
    
    return result

## Understanding working of trackers in OpenCV


Useful websites and places to find codebases:
    - [https://www.pyimagesearch.com/2018/07/30/opencv-object-tracking/] (https://www.pyimagesearch.com/2018/07/30/opencv-object-tracking/)
    - [https://www.learnopencv.com/object-tracking-using-opencv-cpp-python/] (https://www.learnopencv.com/object-tracking-using-opencv-cpp-python/)

Code from link 2 (find learnopencv repo on github and locally at ~):
    
    Do init for a frame and then simply get the tracked box using tracker.update(frame)
    
Code from link 1 (adrian pyimagesearch):
    
    

Observation: KCF tracker is working out to be well.

Get the labels for detected faces in some frames of the video -> get accuracy
- Identify the frames in succession in which faces are not detected, and use the previously labelled frames to make new predictions -> get accuracy again
- Identify the frames in succession in which faces are not detected, take a window, and detect faces again
- Identify the frames in succession in which faces are not detected, take a window, and detect faces again after normalising that window

In [14]:
def processVideo(path_video_frames_folder, dictBox):
    """
    Get dictionary of the form {Video_name: {frame_1:[box_1 ie l,t,w,h], frame_2:[box_2, box_3]}} 
    after updating with tracked boxes (for a single face now.)    
    """
    
    #initiate the tracker
    tracker = cv2.TrackerKCF_create()
    
    # get name of video
    name_video = os.path.basename(path_sample_video_folder2).rsplit(' ',3)[0]
#     print(name_video)
    # get dict frame no: box
    dictFrameNoToBoxes = dictBox[name_video]
#     print(dictFrameNoToBoxes)
    tracking = False
    
    # when frame no comes,check if detection(dict[frame_no] has a box): if not,track and get a box 
        # Save this box
        # Detect face inside the box obtained after resizing this box
            # Save this box
        # normalise this window and detect face.
    
    # Get path of all images in this video and sort them, so that video frames are read in order
    list_image_files = glob.glob(path_sample_video_folder + '/*')
    list_image_files.sort()
#     print(len(list_image_files))
    for frame_no, path_file in enumerate(list_image_files,start=1):
        
#         print("Starting frame number ", frame_no)
        #read the image in grayscale
        frame = cv2.imread(path_file,0)
        
        #list of boxes eg - [[l,t,w,h], [l,t,w,h]]
        if frame_no in dictFrameNoToBoxes:
            boxes_in_this_frame = dictFrameNoToBoxes[frame_no]
        else:
            boxes_in_this_frame = []
#         print("Boxes in this frame", boxes_in_this_frame)
        
        # if the frame has atleast 1 box detected
        if len(boxes_in_this_frame) > 0:
            
            ### check and mark the relevant box, convert to float as reqd by init
            # adjust for multiple faces.
            box_main = (boxes_in_this_frame[0])
            box_main = tuple([float(i) for i in box_main])
            
            # reinitialise the tracker:
            ok = tracker.init(frame, box_main)
            tracking = True
        
        # else - when no boxes detected 
        else:
            # if tracking, update
            if tracking:
                ok, tracked_box = tracker.update(frame)
#                 print("Tracking", ok, tracked_box)
                
                #writing to dict
                tracked_box_int = list(int(i) for i in tracked_box)
                dictBox[name_video][frame_no] = [tracked_box_int]
                
            # if not tracking, don't update
            else:
#                 print("Not Tracking")
                pass
    
    return dictBox

### Update dictionary for all videos

In [15]:
def getUpdatedDictBoxFromXML(path_xml, path_folder_of_folders):
    """
    Get updated dictionary of the form {Video_name: {frame_1:[box_1 ie l,t,w,h], frame_2:[box_2, box_3]}} 
    after tracking updates.
    
    Parameters
    ----------
    path_xml : str
        Path to the original XML (with the frame rectangles detected by a detector)
        
    path_folder_of_folders: str
        Path to the folder having folders of Video frames.
        
    Returns
    -------
    dictionary of dictionary of lists
        Returns the dictionary of the form {Video_name: {frame_1:[box_1 ie l,t,w,h], frame_2:[box_2, box_3]}}  
        with the updated boxes after tracking.
        
    """
    
    updated_dict_box = getDictVideoToFrameNoBoxFromXML(path_xml)
    list_video_names = updated_dict_box.keys()
    list_path_folder_video_frames = glob.glob(path_folder_of_folders + "/*")
    list_path_video_folder = []
    
    # Make a list of path of video folders which appear in the dictionary.
    for video_name in list_video_names:
        for path_folder in list_path_folder_video_frames:
            if video_name in path_folder:
                list_path_video_folder.append(path_folder)
                break
    
    print(len(list_path_video_folder), "folders are being looked at")

    # Update the dictionary after processing all the videos
    for path_video_folder in tqdm(list_path_video_folder):
        updated_dict_box = processVideo(path_video_folder,updated_dict_box)
    
    return updated_dict_box

In [204]:
final_dict_box = getUpdatedDictBoxFromXML(path_xml_dlib_output,path_folder_of_folders)



0it [00:00, ?it/s]

26509it [00:00, 261781.09it/s]

37556it [00:00, 130325.37it/s]

39016it [00:00, 126369.70it/s]

  0%|          | 0/233 [00:00<?, ?it/s]

(233, 'folders')




  0%|          | 1/233 [00:06<23:58,  6.20s/it]

  1%|          | 2/233 [00:09<18:15,  4.74s/it]

  1%|▏         | 3/233 [00:12<16:31,  4.31s/it]

  2%|▏         | 4/233 [00:16<15:29,  4.06s/it]

  2%|▏         | 5/233 [00:19<15:02,  3.96s/it]

  3%|▎         | 6/233 [00:23<14:43,  3.89s/it]

  3%|▎         | 7/233 [00:26<14:22,  3.82s/it]

  3%|▎         | 8/233 [00:30<14:12,  3.79s/it]

  4%|▍         | 9/233 [00:33<13:57,  3.74s/it]

  4%|▍         | 10/233 [00:38<14:10,  3.81s/it]

  5%|▍         | 11/233 [00:41<14:05,  3.81s/it]

  5%|▌         | 12/233 [00:45<13:59,  3.80s/it]

  6%|▌         | 13/233 [00:50<14:20,  3.91s/it]

  6%|▌         | 14/233 [00:55<14:22,  3.94s/it]

  6%|▋         | 15/233 [00:59<14:24,  3.97s/it]

  7%|▋         | 16/233 [01:03<14:17,  3.95s/it]

  7%|▋         | 17/233 [01:06<14:06,  3.92s/it]

  8%|▊         | 18/233 [01:10<13:59,  3.90s/it]

  8%|▊         | 19/233 [01:13<13:48,  3.87s/it]

  9%|▊         | 20/233 [01:16<13:39,  3.85s/it]

  9%|▉ 

### Write a new XML 

In [205]:
def writeXMLFromDictBox(updated_dict_box, path_folder_destination):
    """
    takes the dictionary and saves an XML file for analysis
    """
    
    root = ET.Element("dataset")
    name = ET.SubElement(root, "name").text = "Labelled faces"
    comment = ET.SubElement(root, "comment").text = "These are labelled images from Bagamoyo"
    images = ET.SubElement(root, "images")
    
    # make a filename for each video name
    for video_name in tqdm(updated_dict_box.keys()):
        dict_framenum_to_listboxes = updated_dict_box[video_name]
        frame_numbers = dict_framenum_to_listboxes.keys()
        frame_numbers.sort()
        for frame_num in frame_numbers:
#             xml append image
            image_file_name = video_name + " " + "{0:0=3d}".format(frame_num)
            image_node = ET.SubElement(images, "image", file=image_file_name)
            for box in dict_framenum_to_listboxes[frame_num]:
#                 create box, add to 
                l,t,w,h = box[0],box[1],box[2],box[3]
                box = ET.SubElement(image_node, "box", height=str(h), left=str(l), top=str(t), width=str(w))
    
        tree = ET.ElementTree(root)
        dest_name = path_folder_destination + video_name + '.xml'
        tree.write(dest_name)
        print("xml written to {}".format(dest_name))

In [206]:
writeXMLFromDictBox(final_dict_box,'output_XML/tracking/')



  0%|          | 0/233 [00:00<?, ?it/s]

  3%|▎         | 8/233 [00:00<00:03, 72.46it/s]

xml written to output_XML/tracking/VIDEO_1526475571181 bored_20180516_104528_1803080270.xml
xml written to output_XML/tracking/VIDEO_1526475571181 bored_20180516_110804_731160236.xml
xml written to output_XML/tracking/VIDEO_10-male-confusion_20180517_050147_1758874434.xml
xml written to output_XML/tracking/VIDEO_1526469405985 make a face like this one_20180516_105329_242798671.xml
xml written to output_XML/tracking/VIDEO_06-female-confusion_20180516_110221_101963203.xml
xml written to output_XML/tracking/VIDEO_1526469405985 make a face like this one_20180516_110124_368117096.xml
xml written to output_XML/tracking/VIDEO_03-female-delight_20180516_104912_855688121.xml
xml written to output_XML/tracking/VIDEO_1526469405985 make a face like this one_20180517_051046_318795747.xml
xml written to output_XML/tracking/VIDEO_1526475571181 bored_20180516_105909_1662154728.xml
xml written to output_XML/tracking/VIDEO_00-male-surprise_20180516_091551_1272027070.xml
xml written to output_XML/trackin



  6%|▌         | 13/233 [00:00<00:04, 52.40it/s]

  7%|▋         | 16/233 [00:00<00:04, 44.73it/s]

xml written to output_XML/tracking/VIDEO_10-male-confusion_20180516_091825_514567706.xml
xml written to output_XML/tracking/VIDEO_1526475586855 confused_20180516_104514_1700391713.xml
xml written to output_XML/tracking/VIDEO_08-female-frustration_20180516_110052_2081001093.xml
xml written to output_XML/tracking/VIDEO_00-male-surprise_20180516_091859_1464474503.xml
xml written to output_XML/tracking/VIDEO_1526475571181 bored_20180516_105424_1564488712.xml
xml written to output_XML/tracking/VIDEO_03-female-delight_20180516_110715_1869255788.xml




  8%|▊         | 19/233 [00:00<00:05, 39.53it/s]

  9%|▉         | 22/233 [00:00<00:05, 35.96it/s]

...

xml written to output_XML/tracking/VIDEO_1526475586855 confused_20180516_110810_1142721593.xml




100%|█████████▉| 232/233 [01:06<00:00,  3.50it/s]

xml written to output_XML/tracking/VIDEO_00-male-surprise_20180516_091541_1754978908.xml




100%|██████████| 233/233 [01:06<00:00,  3.48it/s]



xml written to output_XML/tracking/VIDEO_1526469405985 make a face like this one_20180517_045849_1455237136.xml


## Main

In [143]:
# def main():
dictBox = getDictVideoToFrameNoBoxFromXML(path_xml_dlib_output)

39016it [00:00, 176220.97it/s]


In [144]:
path_sample_video_folder2

'/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_1526475571181 bored_20180516_103925_356651116 (5-29-2018 10-06-12 AM)'

In [161]:
processVideo(path_sample_video_folder2,dictBox)
# main()

VIDEO_1526475571181 bored_20180516_103925_356651116
369
('Starting frame number ', 1)
('Boxes in this frame', [['1170', '92', '462', '462']])
('Starting frame number ', 2)
('Boxes in this frame', [['1170', '92', '462', '462']])
('Starting frame number ', 3)
('Boxes in this frame', [[1170, 92, 462, 462]])
('Starting frame number ', 4)
('Boxes in this frame', [['1489', '770', '36', '36']])
('Starting frame number ', 5)
('Boxes in this frame', [[1170, 92, 462, 462]])
('Starting frame number ', 6)
('Boxes in this frame', [[1170, 92, 462, 462]])
('Starting frame number ', 7)
('Boxes in this frame', [['1272', '40', '462', '462']])
('Starting frame number ', 8)
('Boxes in this frame', [['204', '333', '385', '385']])
('Starting frame number ', 9)
('Boxes in this frame', [['1324', '40', '462', '462'], ['247', '333', '385', '385']])
('Starting frame number ', 10)
('Boxes in this frame', [['1324', '40', '462', '462']])
('Starting frame number ', 11)
('Boxes in this frame', [['1375', '40', '462', 

('Boxes in this frame', [['1274', '33', '385', '385']])
('Starting frame number ', 95)
('Boxes in this frame', [['1272', '-62', '462', '513']])
('Starting frame number ', 96)
('Boxes in this frame', [['1272', '-62', '462', '513']])
('Starting frame number ', 97)
('Boxes in this frame', [['1272', '-62', '462', '513']])
('Starting frame number ', 98)
('Boxes in this frame', [[1172, 94, 462, 462]])
('Starting frame number ', 99)
('Boxes in this frame', [['1317', '-52', '385', '428']])
('Starting frame number ', 100)
('Boxes in this frame', [['161', '376', '385', '385'], ['1317', '-52', '385', '428']])
('Starting frame number ', 101)
('Boxes in this frame', [[1172, 92, 462, 462]])
('Starting frame number ', 102)
('Boxes in this frame', [['1324', '-62', '462', '513'], ['142', '348', '463', '463']])
('Starting frame number ', 103)
('Boxes in this frame', [[1172, 94, 462, 462]])
('Starting frame number ', 104)
('Boxes in this frame', [['204', '376', '385', '385']])
('Starting frame number ', 

('Boxes in this frame', [['1317', '33', '385', '385'], ['204', '461', '385', '385']])
('Starting frame number ', 175)
('Boxes in this frame', [['204', '461', '385', '385'], ['598', '669', '321', '321'], ['1317', '33', '385', '385']])
('Starting frame number ', 176)
('Boxes in this frame', [['204', '461', '385', '385'], ['1324', '40', '462', '462'], ['598', '669', '321', '321']])
('Starting frame number ', 177)
('Boxes in this frame', [['1317', '33', '385', '385']])
('Starting frame number ', 178)
('Boxes in this frame', [['1317', '33', '385', '385'], ['142', '400', '463', '462']])
('Starting frame number ', 179)
('Boxes in this frame', [['1317', '33', '385', '385']])
('Starting frame number ', 180)
('Boxes in this frame', [['1317', '33', '385', '385']])
('Starting frame number ', 181)
('Boxes in this frame', [['1272', '-62', '462', '513']])
('Starting frame number ', 182)
('Boxes in this frame', [['1272', '-62', '462', '513']])
('Starting frame number ', 183)
('Boxes in this frame', [[

('Tracking', True, (1174.0, 92.0, 462.0, 462.0))
('Starting frame number ', 260)
('Boxes in this frame', [])
('Tracking', True, (1172.0, 92.0, 462.0, 462.0))
('Starting frame number ', 261)
('Boxes in this frame', [])
('Tracking', True, (1174.0, 92.0, 462.0, 462.0))
('Starting frame number ', 262)
('Boxes in this frame', [])
('Tracking', True, (1172.0, 92.0, 462.0, 462.0))
('Starting frame number ', 263)
('Boxes in this frame', [])
('Tracking', True, (1174.0, 92.0, 462.0, 462.0))
('Starting frame number ', 264)
('Boxes in this frame', [])
('Tracking', True, (1172.0, 92.0, 462.0, 462.0))
('Starting frame number ', 265)
('Boxes in this frame', [])
('Tracking', True, (1174.0, 92.0, 462.0, 462.0))
('Starting frame number ', 266)
('Boxes in this frame', [])
('Tracking', True, (1172.0, 92.0, 462.0, 462.0))
('Starting frame number ', 267)
('Boxes in this frame', [])
('Tracking', True, (1174.0, 92.0, 462.0, 462.0))
('Starting frame number ', 268)
('Boxes in this frame', [])
('Tracking', True, 

('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 340)
('Boxes in this frame', [])
('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 341)
('Boxes in this frame', [])
('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 342)
('Boxes in this frame', [])
('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 343)
('Boxes in this frame', [])
('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 344)
('Boxes in this frame', [])
('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 345)
('Boxes in this frame', [])
('Tracking', True, (1190.0, 86.0, 462.0, 462.0))
('Starting frame number ', 346)
('Boxes in this frame', [])
('Tracking', True, (1192.0, 88.0, 462.0, 462.0))
('Starting frame number ', 347)
('Boxes in this frame', [])
('Tracking', True, (1194.0, 88.0, 462.0, 462.0))
('Starting frame number ', 348)
('Boxes in this frame', [])
('Tracking', True, 

{'VIDEO_00-male-surprise_20180516_091529_219393302': {1: [['-137',
    '295',
    '616',
    '555'],
   ['1060', '675', '385', '385']],
  2: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  3: [['1016', '605', '462', '462'], ['-63', '348', '514', '463']],
  4: [['-137', '295', '616', '555'], ['1016', '605', '462', '462']],
  5: [['-137', '295', '616', '555'], ['1016', '605', '462', '462']],
  6: [['-137', '295', '616', '555'], ['1016', '605', '462', '462']],
  7: [['-137', '295', '616', '555'], ['1016', '605', '462', '462']],
  8: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  9: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  10: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  11: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  12: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  13: [['-63', '348', '514', '463'], ['1016', '605', '462', '462']],
  14: [['-63', '348', '514', '463'], ['1

In [136]:
dictBox['VIDEO_1526475571181 bored_20180516_103925_356651116']

{1: [['1170', '92', '462', '462']],
 2: [['1170', '92', '462', '462']],
 3: [],
 4: [['1489', '770', '36', '36']],
 5: [],
 6: [],
 7: [['1272', '40', '462', '462']],
 8: [['204', '333', '385', '385']],
 9: [['1324', '40', '462', '462'], ['247', '333', '385', '385']],
 10: [['1324', '40', '462', '462']],
 11: [['1375', '40', '462', '462'], ['290', '376', '385', '385']],
 12: [['1375', '40', '462', '462'], ['332', '418', '386', '386']],
 13: [['1375', '40', '462', '462'], ['332', '418', '386', '386']],
 14: [['290', '418', '385', '386']],
 15: [['290', '418', '385', '386']],
 16: [['290', '418', '385', '386'], ['1324', '40', '462', '462']],
 17: [['1324', '40', '462', '462'], ['247', '418', '385', '386']],
 18: [['247', '418', '385', '386'], ['1281', '49', '554', '554']],
 19: [['1272', '92', '462', '462'], ['194', '400', '462', '462']],
 20: [['1272', '92', '462', '462']],
 21: [['1272', '92', '462', '462'], ['204', '461', '385', '385']],
 22: [['204', '461', '385', '385'], ['1272', '9

In [150]:
def processVideoMultipleObjects(path_video_frames_folder, dictBox):
    """
    Get information for when the video is running.
    """
    
    #initiate the tracker
    tracker = cv2.TrackerKCF_create()
    
    # get name of video
    name_video = os.path.basename(path_sample_video_folder2).rsplit(' ',3)[0]
    print(name_video)
    # get dict frame no: box
    dictFrameNoToBoxes = dictBox[name_video]
#     print(dictFrameNoToBoxes)
    tracking = False
    
    # when frame no comes,check if detection(dict[frame_no] has a box): if not,track and get a box 
        # Save this box
        # Detect face inside the box obtained after resizing this box
            # Save this box
        # normalise this window and detect face.
    
    # Get path of all images in this video
    list_image_files = glob.glob(path_sample_video_folder + '/*')
    list_image_files.sort()
    print(len(list_image_files))
    for frame_no, path_file in enumerate(list_image_files,start=1):
        
        print("Starting frame number ", frame_no)
        # 
        frame = cv2.imread(path_file,0)
        
        #list of boxes eg - [[l,t,w,h], [l,t,w,h]]
        boxes_in_this_frame = dictFrameNoToBoxes[frame_no]
        print("Boxes in this frame", boxes_in_this_frame)
        
        # if the frame has a box detected, reinitialise the tracker
        if len(boxes_in_this_frame) > 0:
            
            ### check and mark the relevant box, convert to float as reqd by init
            box_main = (boxes_in_this_frame[0])
            box_main = tuple([float(i) for i in box_main])
            
            # reinitialise the tracker:
            ok = tracker.init(frame, box_main)
            tracking = True
        
        # else - when no boxes detected, track and update the bounding box
        else:
            # if tracking, update
            if tracking:
                ok, tracked_box = tracker.update(frame)
                print("Tracking", ok, tracked_box)
                
                #writing to dict
                tracked_box_int = tuple(int(i) for i in tracked_box)
                print("int",tracked_box_int)
            # if not tracking, don't update
            else:
                print("Not Tracking")

# Debug

In [11]:
processVideo(path_folder_videos + "VIDEO_male-delight2_20180517_111853_1366476353.mp4",1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171


In [12]:
cap = cv2.VideoCapture(path_sample_video)
property_id = int(cv2.CAP_PROP_FRAME_COUNT) 
length = int(cv2.VideoCapture.get(cap, property_id))
print( length )

173


In [ ]:
path_folder_videos